<a href="https://colab.research.google.com/github/Keita062/work/blob/main/Eye_11_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from datetime import datetime, timedelta

In [11]:
Eye = pd.read_csv('/content/DO.csv')

In [12]:
Eye = Eye.drop(index=range(0, 218))
Eye = Eye.drop(index=range(2878, 3162))

Eye.head()

,DataCount,TIME(Y/M/D h:m:s),milliSeconds,leftX,leftY,rightX,rightY,leftPSize,rightPSize
218,218,2024/11/02 18:13:44,744,1076.826,491.9123,1053.470,510.0161,23.4658,24.2151
219,219,2024/11/02 18:13:44,769,1077.004,506.2887,1061.180,523.4087,23.2076,23.8953
220,220,2024/11/02 18:13:44,800,1077.004,506.2887,1061.180,523.4087,23.2076,23.8953
221,221,2024/11/02 18:13:44,846,1074.611,548.1039,1088.470,595.0340,22.3527,23.1011
222,222,2024/11/02 18:13:44,863,1096.853,540.6955,1080.904,536.4927,20.8244,21.4515


In [13]:
want = Eye.iloc[:, 2:9]
want.head()
# 7列中4列以上が0の行をすべて0にする
want[(want == 0).sum(axis=1) >= 4] = 0
# 7列中4列未満が0の行で0でない前の行まで遡って補完
for i in range(1, len(want)):  # 2行目以降で実行
    if (want.iloc[i] == 0).sum() < 4:  # 0が4列未満の行のみ
        for col in want.columns:  # 各列ごとに補完を確認
            if want.iloc[i][col] == 0:
                # 0でない値を見つけるまで遡って補完
                j = i - 1
                while j >= 0 and want.iloc[j][col] == 0:
                    j -= 1
                if j >= 0:  # 0でない値が見つかった場合
                    want.at[i, col] = want.iloc[j][col]  # 0でない値を設定

# 3行以上連続して0がある部分を「まばたき」として検出
blink_rows = []
zero_count = 0

for i in range(len(want)):
    if (want.iloc[i] == 0).all():  # 行がすべて0の場合
        zero_count += 1
    else:
        if zero_count >= 3:
            blink_rows.append((i - zero_count, i - 1))  # まばたきの行範囲を記録
        zero_count = 0

# 最後の行が連続する0で終わっていた場合も処理
if zero_count >= 3:
    blink_rows.append((len(want) - zero_count, len(want) - 1))

print("Blinks detected at rows:", blink_rows)

# 瞬きの数をカウントする変数
blink_count = 0
zero_count = 0  # 連続して0が続いている行数

# 行ごとにすべての列が0かを確認
for i in range(len(want)):
    if (want.iloc[i] == 0).all():  # 行のすべての列が0の場合
        zero_count += 1
    else:
        if zero_count >= 3:  # 3行以上連続して0が続いていた場合
            blink_count += 1
        zero_count = 0  # 連続カウントをリセット

# データフレームの最後まで0が連続していた場合のチェック
if zero_count >= 3:
    blink_count += 1

print("瞬きの数:", blink_count)


want["gazeX"] = (want["leftX"] + want["rightX"]) / 2
want["gazeY"] = (want["leftY"] + want["rightY"]) / 2

use = Eye.iloc[:, [0, 1]]
result = pd.concat([use,want],axis=1)

result.head()


Blinks detected at rows: [(50, 57), (142, 147), (180, 186), (539, 542), (648, 651), (852, 856), (1037, 1043), (1224, 1226), (1432, 1438), (1530, 1536), (1550, 1557), (1719, 1723), (1779, 1783), (1964, 1968), (2113, 2116), (2299, 2305)]
瞬きの数: 16


,DataCount,TIME(Y/M/D h:m:s),milliSeconds,leftX,leftY,rightX,rightY,leftPSize,rightPSize,gazeX,gazeY
218,218.0,2024/11/02 18:13:44,744.0,1076.826,491.9123,1053.470,510.0161,23.4658,24.2151,1065.1480,500.96420
219,219.0,2024/11/02 18:13:44,769.0,1077.004,506.2887,1061.180,523.4087,23.2076,23.8953,1069.0920,514.84870
220,220.0,2024/11/02 18:13:44,800.0,1077.004,506.2887,1061.180,523.4087,23.2076,23.8953,1069.0920,514.84870
221,221.0,2024/11/02 18:13:44,846.0,1074.611,548.1039,1088.470,595.0340,22.3527,23.1011,1081.5405,571.56895
222,222.0,2024/11/02 18:13:44,863.0,1096.853,540.6955,1080.904,536.4927,20.8244,21.4515,1088.8785,538.59410


In [14]:
result.head()

,DataCount,TIME(Y/M/D h:m:s),milliSeconds,leftX,leftY,rightX,rightY,leftPSize,rightPSize,gazeX,gazeY
218,218.0,2024/11/02 18:13:44,744.0,1076.826,491.9123,1053.470,510.0161,23.4658,24.2151,1065.1480,500.96420
219,219.0,2024/11/02 18:13:44,769.0,1077.004,506.2887,1061.180,523.4087,23.2076,23.8953,1069.0920,514.84870
220,220.0,2024/11/02 18:13:44,800.0,1077.004,506.2887,1061.180,523.4087,23.2076,23.8953,1069.0920,514.84870
221,221.0,2024/11/02 18:13:44,846.0,1074.611,548.1039,1088.470,595.0340,22.3527,23.1011,1081.5405,571.56895
222,222.0,2024/11/02 18:13:44,863.0,1096.853,540.6955,1080.904,536.4927,20.8244,21.4515,1088.8785,538.59410


In [15]:
result.columns

Index(['DataCount', 'TIME(Y/M/D h:m:s)', ' milliSeconds', 'leftX', 'leftY',
       'rightX', 'rightY', 'leftPSize', 'rightPSize', 'gazeX', 'gazeY'],
      dtype='object')

In [18]:
result.dtypes

,0
DataCount,float64
TIME(Y/M/D h:m:s),object
milliSeconds,float64
leftX,float64
leftY,float64
rightX,float64
rightY,float64
leftPSize,float64
rightPSize,float64
gazeX,float64


In [63]:
# ヒートマップのアルファブレンディング関数
def overlay_heatmap(frame, heatmap, alpha=0.6):
    # ヒートマップをカラーマップに変換
    heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    # サイズをフレームにリサイズ
    heatmap_color = cv2.resize(heatmap_color, (frame.shape[1], frame.shape[0]))
    # フレームとヒートマップをブレンド
    overlay = cv2.addWeighted(heatmap_color, alpha, frame, 1 - alpha, 0)
    return overlay


In [65]:
result['TIME'] = pd.to_datetime(result['TIME(Y/M/D h:m:s)'])  # datetime型に変換

In [66]:
# y 軸を反転（画面の左上を [0, 0]、右下を [1920, 1080] に設定）
result['gazeY'] = 1080 - result['gazeY']

In [57]:
result.head()

,DataCount,TIME(Y/M/D h:m:s),milliSeconds,leftX,leftY,rightX,rightY,leftPSize,rightPSize,gazeX,gazeY,TIME
218,218.0,2024-11-02 18:13:44,744.0,1076.826,491.9123,1053.470,510.0161,23.4658,24.2151,1065.1480,500.96420,2024-11-02 18:13:44
219,219.0,2024-11-02 18:13:44,769.0,1077.004,506.2887,1061.180,523.4087,23.2076,23.8953,1069.0920,514.84870,2024-11-02 18:13:44
220,220.0,2024-11-02 18:13:44,800.0,1077.004,506.2887,1061.180,523.4087,23.2076,23.8953,1069.0920,514.84870,2024-11-02 18:13:44
221,221.0,2024-11-02 18:13:44,846.0,1074.611,548.1039,1088.470,595.0340,22.3527,23.1011,1081.5405,571.56895,2024-11-02 18:13:44
222,222.0,2024-11-02 18:13:44,863.0,1096.853,540.6955,1080.904,536.4927,20.8244,21.4515,1088.8785,538.59410,2024-11-02 18:13:44


In [67]:
# 動画読み込み
video_path = "/content/TVアニメ「ダンダダン」オープニング映像｜Creepy Nuts「オトノケ」 - MBSアニメ&ドラマ (1080p, h264).mp4"
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [68]:
# 動画の開始時間を指定（例: データの最初の時間）
video_start_time = result['TIME'].min()

In [69]:
# 出力動画設定
out = cv2.VideoWriter('output_with_heatmap.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

In [70]:
frame_index = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # フレームに対応する時間範囲を計算
    frame_time = video_start_time + timedelta(seconds=frame_index / fps)
    next_frame_time = frame_time + timedelta(seconds=1 / fps)

    # 時間範囲に対応する視線データを取得
    gaze_points = result[(result['TIME'] >= frame_time) & (result['TIME'] < next_frame_time)][['gazeX', 'gazeY']].values

    if len(gaze_points) > 0:
        # 視線ポイントをヒートマップに変換
        heatmap, _, _ = np.histogram2d(
            gaze_points[:, 1], gaze_points[:, 0], bins=[frame_height, frame_width],
            range=[[0, 1080], [0, 1920]]
        )
        heatmap = (heatmap / heatmap.max() * 255).astype(np.uint8)

        # フレームにヒートマップを重ねる
        frame = overlay_heatmap(frame, heatmap)

    out.write(frame)
    frame_index += 1


In [71]:
# リソース解放
cap.release()
out.release()